In [ ]:
import json
import os
import shutil
import pandas as pd
from pathlib import Path
from time import sleep
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

dirpath = os.getcwd()+'\\chromedriver.exe' 
download_path = os.getcwd()+'\\output'
chrome_options = Options()
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument("--enable-javascript")

appState = {
   "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
prefs = {
    'printing.print_preview_sticky_settings.appState': json.dumps(appState),
    'savefile.default_directory': download_path
        }

chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--kiosk-printing')

browser = webdriver.Chrome(executable_path=dirpath, options=chrome_options) #instancia o driver

In [109]:
#browser.get("https://www.rappi.com.br/lojas/900154970-grupo-big-hiper-bompreco-recife/ofertas/cerveja-e-destilados")
#browser.get("https://www.rappi.com.br/lojas/900154970-grupo-big-hiper-bompreco-recife/ofertas/mercearia")
browser.get("https://www.rappi.com.br/lojas/900154970-grupo-big-hiper-bompreco-recife/ofertas/acougue-e-peixaria")

In [110]:
dic = {"Produto": [],
       "Preço": [],
       "Desconto %": [],
       "Quantidade": []
}

In [115]:
#itens = browser.find_elements_by_xpath("*//h2[contains(text(), 'Cerveja e destilados' )]//..//..//div[2]//ul//div")
#itens = browser.find_elements_by_xpath("*//h2[contains(text(), 'Mercearia' )]//..//..//div[2]//ul//div")
itens = browser.find_elements_by_xpath("*//h2[contains(text(), 'Açougue e peixaria' )]//..//..//div[2]//ul//div")

In [117]:
porcentagem = []
for x in itens:
    if re.search("%", x.text) and len(x.text.split("\n")) >=4:
        porcentagem.append(x)

        
for x in porcentagem:
    list_temp = x.text.split("\n")
    if len(list_temp) == 6:
        list_temp.remove(list_temp[2])
        list_temp.remove(list_temp[0])
    else:
        list_temp.remove(list_temp[1])
        
    dic["Produto"].append(list_temp[2])
    dic["Preço"].append(list_temp[0])
    dic["Desconto %"].append(list_temp[1])
    dic["Quantidade"].append(list_temp[3])
    
df = pd.DataFrame(dic).drop_duplicates(ignore_index=True)
df['Desconto %'] = df['Desconto %'].str.replace("%", '')
df['Desconto %'] = df['Desconto %'].astype('float64')
df = df.sort_values('Desconto %', ascending=False, ignore_index=True)

In [118]:
df

,Produto,Preço,Desconto %,Quantidade
0,Acem Bovino Sem Osso,"R$ 28,90/kg",31.0,1 X 1 Kg
1,Bisteca Suína Congelada,"R$ 12,79/kg",27.0,1 X 1 Kg
2,Sadia Bio Sobrecoxa De Frango,"R$ 15,99",20.0,1 X 1 Kg
3,Seara Filé Frango,"R$ 26,99",17.0,1 X 1 Kg
4,Masterboi Carne Moída Acém,"R$ 25,90",16.0,Aprox. 500g/ud
5,Sadia Filé de Frango de Peito Congelado Sem Osso,"R$ 25,99",15.0,1 X 1 Kg
6,Perdigão Filezinho de Frango,"R$ 23,99",11.0,1 X 1 Kg
7,Seara Sassami Frango Filezinho,"R$ 24,99",9.0,1 X 1 Kg
8,Perdigão Filezinho De Frango Congelado,"R$ 24,99",7.0,1 X 1 Kg
9,Sadia Filezinho Sassami de Frango Congelado,"R$ 25,99",5.0,1 X 1 Kg
